<div class="alert alert-block alert-info">
<h3>Student Information</h3> Please provide information about yourself. We will NOT grade this submissing w/o all the information<br>
<b>Name</b>: Avantika Yellapantula<br>
<b>NetID</b>:ay257<br> 
<b>Recitation (01/02)</b>: 01<br>
<b>Notes to Grader</b> (optional):<br>
<br><br>
<b>IMPORTANT</b>
Your work will not be graded withour your initials below<br>
I certify that this lab represents my own work and I have read the RU academic intergrity policies at<br>
<a href="https://www.cs.rutgers.edu/academic-integrity/introduction">https://www.cs.rutgers.edu/academic-integrity/introduction </a><br>
<b>Initials</b>: AY     (eg: DS for Deeptanshu Singh)


<h3>Grader Notes</h3>
<b>Your Grade<b>:<br>
<b>Grader Initials</b>:<br>
<b>Grader Comments</b> (optional):<br>
</div>

## CS 439 - Introduction to Data Science
### Spring 2019

# Lab 3: Data Cleaning and Visualization

### Due Date: Sunday February 17, 2019 by 11:59 PM ###

### Instructions
This lab is presented as a notebook. Please execute the cells that are already completed and your task is to fill in the code
between ### BEGIN SOLUTION ### and ### END SOLUTION ###. 

#### Important: Please do not add any new cells or change the order of cells. If you have questions, please contact the courseS staff.

In this lab, you will be working with a dataset from NYPD containing data on calls to the New York Police Department. Information about the datasets can be found https://opendata.cityofnewyork.us/

## Setup

Note that after activating matplotlib to display figures inline via the IPython magic `%matplotlib inline`, we configure a custom default figure size. Virtually every default aspect of matplotlib [can be customized](https://matplotlib.org/users/customizing.html).

In [1]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

# Part 1:Getting Data

We will work with the NYPD Historic complaint data set. Our first task is to estimate the size of this download by looking at the number of rows, columns and using an estimated size for a column (use a reasonable value). The site metadata is available from the page
https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i

In [2]:
### BEGIN SOLUTION
#The number of rows and columns are available from the meta-data website. There are
#6.04 million rows and 35 columns.

#                                      Num_obs*Vars*Width + 4*Num_obs
#       number of megabytes  =  M  =  --------------------------------
#                                                   1024^2
#
# The number of observations is 6.04M*35=211,400,000
# The number of variables are the number of columns, which is 35
# I will use the width of an integer (4)

# So the math gives us: 5,966,753,388,257.599

### END SOLUTION

### 1.1 Download the data
This file is large (use the estimate you did above). If it takes too long to download, you may want to interrupt and download the file using a browser and URL https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i

In [ ]:
import utils
data_dir = 'data'
data_url = 'https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i'

file_name = 'NYPD_Complaint_Data_Historic.csv'

# To retrieve the dataset, we will use the `utils.fetch_and_cache` utility from utils library. 
dest_path = utils.fetch_and_cache(data_url=data_url, file=file_name,data_dir=data_dir)
print(f'Located at {dest_path}')

### 1.2 Inspect the size of the file
It is helpful to get an idea of the size of the file. This can be done using functions in the utils library.

In [32]:
# look at the size of the file w/o opening it using OS (https://docs.python.org/3/library/os.html). You can perform
# variety of operating system related functions from this package.
### BEGIN SOLUTION
import os
statinfo = os.stat('C:/Users/avocado/Desktop/Spring 2019/DS/Labs/Lab3/Lab3/data/NYPD_Complaint_Data_Historic.csv')
statinfo.st_size
#### END SOLUTION

1898173978

### 1.3 Split the large file
This data file NYPD_Complaint_Data_Historic.csv is too big to load into a single DataFrame. Let us split the large file into smaller files.  Let us find out the number of lines in the NYPD_Complaint_Data_Historic.csv file using utils.

In [ ]:
# Using utils library, find the number of lines in the file
from utils import line_count
print("Lines in the calls file", line_count('C:/Users/avocado/Desktop/Spring 2019/DS/Labs/Lab3/Lab3/data/NYPD_Complaint_Data_Historic.csv'))
#%ls

### END SOLUTION

In [5]:
# Split the file into 10 smaller files. Estimate the number of lines in each file using the count above
# files should be created in the data folder and named NYPD_Complaint_Data_Historic_1.csv, 
# NYPD_Complaint_Data_Historic_2.csv, ... NYPD_Complaint_Data_Historic_10.csv etc
# It is possible that few lines from the original file may not be saved due to rounding errors.

### BEGIN SOLUTION
# import sys
# orig_file = 'data/NYPD_Complaint_Data_Historic.csv'
# csvfilename = open('data/NYPD_Complaint_Data_Historic.csv', 'r').readlines()
# lines_per_file = 100000  # gives about 25 MB
# file = 1
# for i in range(len(csvfilename)):
#    if i % lines_per_file == 0:
#      open(str(orig_file)+ str(file) + '.csv', 'w+').writelines(csvfilename[i:i+lines_per_file])
#    file += 1
# # this actually creates more managable files of size 25MB each


# above code doesn't work for shit because the file is over 1gb
# can't just use open(filename,'r') either because that would say that the type_IO error
# is subscriptable.

import os

def split(filehandler, delimiter=',', row_limit=604000,
          output_name_template='NYPD_Complaint_Data_Historic_%s.csv', output_path='C:/Users/avocado/Desktop/Spring 2019/DS/Labs/Lab3/Lab3/data', keep_headers=True):
    import csv
    reader = csv.reader(filehandler, delimiter=delimiter)
    current_piece = 1
    current_out_path = os.path.join(
        output_path,
        output_name_template % current_piece
    )
    current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
    current_limit = row_limit
    if keep_headers:
        headers = next(reader)
        current_out_writer.writerow(headers)
    for i, row in enumerate(reader):
        if i + 1 > current_limit:
            current_piece += 1
            current_limit = row_limit * current_piece
            current_out_path = os.path.join(
                output_path,
                output_name_template % current_piece
            )
            current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
            if keep_headers:
                current_out_writer.writerow(headers)
        current_out_writer.writerow(row)

split(open('data/NYPD_Complaint_Data_Historic.csv', 'r'));        

### END SOLUTION

### 1.4 Check the files in the data folder
Now, we'll use a method of the `Pathlib.Path` class called `glob` to list all files in the `data` directory. You will find useful information in pathlib [docs](https://docs.python.org/3/library/pathlib.html).

Below, we use pathlib's `glob` method to store the list of all files' names from the `data_dir` directory in the variable `file_names`. These names should be strings that contain only the file name (e.g. `dummy.txt` not `data/dummy.txt`). The asterisk (*) character is used with the `glob` method to match any string.

In [6]:
from pathlib import Path
data_dir_path = Path('data') # creates a Path object that points to the data directory
file_names = [x.name for x in data_dir_path.glob('*') if x.is_file()]
file_names

['NYPD_Complaint_Data_Historic.csv',
 'NYPD_Complaint_Data_Historic_1.csv',
 'NYPD_Complaint_Data_Historic_10.csv',
 'NYPD_Complaint_Data_Historic_2.csv',
 'NYPD_Complaint_Data_Historic_3.csv',
 'NYPD_Complaint_Data_Historic_4.csv',
 'NYPD_Complaint_Data_Historic_5.csv',
 'NYPD_Complaint_Data_Historic_6.csv',
 'NYPD_Complaint_Data_Historic_7.csv',
 'NYPD_Complaint_Data_Historic_8.csv',
 'NYPD_Complaint_Data_Historic_9.csv']

### 1.5 Pre-processing of data
It is good to pre-process the data to see if the file can be opened in a Jupyter notebook. We need to avoid large files that can crash notebooks. Typically, files of size around 200 MB is ok to open into a DataFrame. In the following activities we will inspect the file w/o opening it as a DataFrame. Using utils.head 

In [7]:
# Use the util.head() to read up to 5 lines from the original file (w/o opening it)
from utils import head
head('data/NYPD_Complaint_Data_Historic.csv')

['CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX\n',
 '756486364,12/10/2013,17:30:00,12/10/2013,22:30:00,52,12/11/2013,107,BURGLARY,223,"BURGLARY,RESIDENCE,NIGHT",COMPLETED,FELONY,BRONX,INSIDE,RESIDENCE - APT. HOUSE,N.Y. POLICE DEPT,0,NA,,NA,1014994,256310,,UNKNOWN,U,,40.870140948,-73.888840951,"(40.870140948, -73.888840951)",PATROL BORO BRONX,,45-64,BLACK HISPANIC,M\n',
 '306416158,12/10/2013,17:30:00,12/10/2013,17:35:00,24,12/11/2013,578,HARRASSMENT 2,638,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,MANHATTAN,INSIDE,DOCTOR/DENTIST OFFICE,N.Y. POLICE DEPT,0,NA,,NA,992349,225599,,WHITE,M,,40.785897843,-73.970755108,"(40.785897843, -73.97075510

### 1.6 Inspecting and describing data columns
There should be 35 columns in each record. Using header information and data types, describe the type of data in each
column. If you are unable to determine, just state so.
##### BEGIN SOLUTION
* CMPLNT_NUM : 
* CMPLNT_FR_DT: 
* CMPLNT_FR_TM:
* CMPLNT_TO_DT:
* CMPLNT_TO:
* ....
* VIC_RACE:
* VIC_SEX:
##### END SOLUTION

# Part 2 - Exploratory Data Analysis
Exploratory data analysis (EDA) is the process of examining a subest of a large data set to see what we can know about the data. First we will explore one file NYPD_Complaint_Data_Historic_1.csv to see what we can find out.

### 2.1 Loading Data into a DataFrame
Load the first CSV file, NYPD_Complaint_Data_Historic_1.csv into a `pandas.DataFrame` object. Also do a time analysis to see how long it took to load the data into a DataFrame. Time should be printed in seconds. The time libraries https://docs.python.org/3/library/time.html can help.

In [15]:
### BEGIN SOLUTION
import pandas as pd
import time
import sys
time_start = time.time()    # returns the number of seconds to current time from 1/1/1970
dfcsv1=pd.read_csv('data/NYPD_Complaint_Data_Historic_1.csv')
time_end = time.time()

elapsed_time = time_end - time_start
print(elapsed_time)

### END SOLUTION

3.1901426315307617


### 2.2 Description of Fields
Let's also check some basic information about these files using the `DataFrame.describe` and `DataFrame.info` methods. Describe columns that can be removed based on the information.

In [17]:
### BEGIN SOLUTION
dfcsv1.info()
dfcsv1.describe()

# What columns can be removed from the DataFrame? 
# A reasonable rule of thumb is that if a column is missing more than 
# 50% of the data then it should be removed

# Going by this rule of thumb, the following data columns should be removed
# PARKS_NM             1042 non-null object
# HADEVELOPT           29918 non-null object
# HOUSING_PSA          46799 non-null float64
# TRANSIT_DISTRICT     12863 non-null float64
# SUSP_AGE_GROUP       113036 non-null object
# STATION_NAME         12863 non-null object

### END SOLUTION

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604000 entries, 0 to 603999
Data columns (total 35 columns):
CMPLNT_NUM           604000 non-null int64
CMPLNT_FR_DT         603956 non-null object
CMPLNT_FR_TM         603998 non-null object
CMPLNT_TO_DT         472813 non-null object
CMPLNT_TO_TM         473035 non-null object
ADDR_PCT_CD          603992 non-null float64
RPT_DT               604000 non-null object
KY_CD                604000 non-null int64
OFNS_DESC            602097 non-null object
PD_CD                603574 non-null float64
PD_DESC              603574 non-null object
CRM_ATPT_CPTD_CD     603999 non-null object
LAW_CAT_CD           604000 non-null object
BORO_NM              603565 non-null object
LOC_OF_OCCUR_DESC    474763 non-null object
PREM_TYP_DESC        601159 non-null object
JURIS_DESC           604000 non-null object
JURISDICTION_CODE    603574 non-null float64
PARKS_NM             1042 non-null object
HADEVELOPT           29918 non-null object
HOUSING_PSA

,CMPLNT_NUM,ADDR_PCT_CD,KY_CD,PD_CD,JURISDICTION_CODE,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,TRANSIT_DISTRICT,Latitude,Longitude
count,6.040000e+05,603992.000000,604000.000000,603574.000000,603574.000000,46799.000000,6.039880e+05,603988.000000,12863.000000,603988.000000,603988.000000
mean,5.506378e+08,63.851955,293.267604,413.477383,0.802303,7076.263360,1.005028e+06,206676.923399,14.129752,40.733914,-73.924998
std,2.597364e+08,34.456046,148.487350,219.521342,6.992332,14596.281384,2.146661e+04,30143.359485,12.403784,0.082740,0.077416
min,1.000024e+08,1.000000,101.000000,101.000000,0.000000,218.000000,9.134630e+05,121131.000000,1.000000,40.498905,-74.254560
25%,3.257399e+08,40.000000,116.000000,254.000000,0.000000,477.000000,9.918910e+05,184360.000000,3.000000,40.672659,-73.972428
50%,5.507877e+08,66.000000,341.000000,386.000000,0.000000,696.000000,1.004535e+06,205190.000000,11.000000,40.729858,-73.926828
75%,7.753875e+08,94.000000,351.000000,637.000000,0.000000,1251.000000,1.016751e+06,234532.000000,30.000000,40.810396,-73.882687
max,9.999986e+08,123.000000,881.000000,969.000000,97.000000,68980.000000,1.067298e+06,271820.000000,34.000000,40.912723,-73.700316


### 2.2 Finding Uniques

Notice that the functions above reveal type information for the columns, as well as some basic statistics about the numerical columns found in the DataFrame. However, we still need more information about what each column represents. Let's explore the data further.

find the number of unique values in each DataFrame column and answer the questions below.

In [18]:
### BEGIN SOLUTION
dfcsv1.nunique()

# Questions
# 1. How many distinct locations where the complaints have come from? 5
# 2. How many age groups are represented in the data set?  6
# 3. How many boroughs are included in the data set? 5
# 4. How many offense types are listed in this data set? 59

### END SOLUTION

CMPLNT_NUM           604000
CMPLNT_FR_DT           2035
CMPLNT_FR_TM           1440
CMPLNT_TO_DT           1391
CMPLNT_TO_TM           1440
ADDR_PCT_CD              77
RPT_DT                  449
KY_CD                    69
OFNS_DESC                59
PD_CD                   366
PD_DESC                 356
CRM_ATPT_CPTD_CD          2
LAW_CAT_CD                3
BORO_NM                   5
LOC_OF_OCCUR_DESC         5
PREM_TYP_DESC            70
JURIS_DESC               20
JURISDICTION_CODE        20
PARKS_NM                348
HADEVELOPT              266
HOUSING_PSA             386
X_COORD_CD            52580
Y_COORD_CD            55685
SUSP_AGE_GROUP            6
SUSP_RACE                 7
SUSP_SEX                  3
TRANSIT_DISTRICT         12
Latitude              77379
Longitude             77382
Lat_Lon               77389
PATROL_BORO               8
STATION_NAME            364
VIC_AGE_GROUP             6
VIC_RACE                  7
VIC_SEX                   4
dtype: int64

### 2.3 Offense by Boro
Using GroupBy operation, create a DataFrame that groups offenses by Boro. call the DataFrame calls_by_Boro_and_offense

In [37]:
### BEGIN SOLUTION
calls_by_Boro_and_offense=dfcsv1.groupby(['OFNS_DESC','BORO_NM'])
# for name,group in calls_by_Boro_and_offense:
#    print(name)
#    print(group)
### END SOLUTION

In [38]:
## Assert that in Bronx there were 6022 Grand Larceny incidents
### BEGIN SOLUTION
assert calls_by_Boro_and_offense['BRONX', 'GRAND LARCENY'] == 6022
### END SOLUTION

KeyError: "Columns not found: 'GRAND LARCENY', 'BRONX'"

### 2.4 Offenses in Bronx

In the cell below, find a list of strings corresponding to the possible values for `OFNS_DESC` when `BORO` is "BRONX". Create an expression that automatically extracts the names of the offenses.

In [50]:
### BEGIN SOLUTION
df_boro=dfcsv1[['BORO_NM','OFNS_DESC']].copy()
for boro,offense in df_boro:
    print(boro)
    print(offense)
### END SOLUTION

# How many offenses were committed in Bronx during the analysis period?

ValueError: too many values to unpack (expected 2)

### 2.5 Most Common Crimes in NYC

What are the five crime types of OFNS_DESC that have the most crime events in Bronx? You may need to use `value_counts` to find the answer. Save your results as a list of strings.

**Hint:** *The `keys` method of the Series class might be useful.*

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

### 2.6 Probability of a Crime in Bronx
What is the probability that a the crime "Arson" can happen in Bronx?

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

---
# Part 3: Visualizing the Data

## Pandas vs. Seaborn Plotting

Pandas offers basic functionality for plotting. For example, the `DataFrame` and `Series` classes both have a `plot` method. However, the basic plots generated by pandas are not particularly pretty. While it's possible to manually use matplotlib commands to make pandas plots look better, we'll instead use a high level plotting library called Seaborn that will take care of most of this for us.

As you learn to do data visualization, you may find the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) and [Seaborn documentation](https://seaborn.pydata.org/api.html) helpful!

### 3.1 Plotting a Series
Using the built-in plotting functionality of pandas, such as `plot` method of the `Series` class to generate a `barh` plot type,  display the value counts for `OFNS_DESC` visually as a barh chart.

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

### 3.2 Getting a Better Plot
The plot above can be messy as it plots all offenses. Plot only the offenses that has more than 10000 calls

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

By contrast, the Seaborn library provides a specific function `countplot` built for plotting counts. It operates directly on the DataFrame itself i.e. there's no need to call `value_counts()` at all. This higher level approach makes it easier to work with. Use the y-label ("Crime Category"), x-label("Number of Calls") and title_of_plot("Number of Calls By Crime Type"

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

You may note that the ordering might be different for the seaborn plot (as compared to pandas plot). If we want the same ordering that we had in the pandas plot, we can use the order parameter of the `countplot` method. It takes a list of strings corresponding to the axis to be ordered. By passing the index of the `value_counts`, you can get the order you want.

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

Now we have a pretty bar plot with the bars ordered by size. Though `seaborn` appears to provide a superior plot from a aesthetic point of view, the `pandas` plotting library is also good to understand. You'll get practice using both libraries in the following questions.

## An Additional Note on Plotting in Jupyter Notebooks

You may have noticed that many of our code cells involving plotting end with a semicolon (;). This prevents any extra output from the last line of the cell that we may not want to see. Try adding this to your own code in the following questions!

### 3.3 making more plots

Now it is your turn to make some more plots using `pandas` and `seaborn`. Let's start by looking at the distribution of calls over days of the week.

The CMPLNT_FR_DT field contains the date of the event. We would like to add a new column to the DataFrame that includes Day of the week (DAY_OF_WEEK) that indicates the day of the week. This can help us analyze the crimes on a specific day of the week. For example, we can answer questions such as "what day of the week that a LARSON is likely to happen in NYC?"


Add a new column `DAY_OF_WEEK` into the `calls` dataframe that has the day string (eg. 'Sunday') for the corresponding value in CMPLNT_FR_DT. For example, if the first 3 values of `CMPLNT_FR_DT` are `['01/27/2006, '01/28/2006, '01/29/2006]`, then the first 3 values of the `DAY_OF_WEEK` column should be `["Friday", "Saturday", "Sunday"]`.

**Hint:** *Try using the [Series.map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function on `calls["OFNS_DESC"]`.  Can you assign this to the new column `calls["DAY_OF_WEEK"]`?*

In [1]:
days = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
### BEGIN SOLUTION


### END SOLUTION

### 3.4 Seaborn plots

Create a `seaborn` plot that shows the number of calls for each day of the week. You may want to use of the `rotation` argument in `ax.set_xticklabels`, which rotates the labels by 90 degrees.

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

Now, let's make the same plot using `pandas`. Construct a vertical bar plot with the count of the number of calls (entries in the table) for each day of the week **ordered by the day of the week** (eg. `Sunday`, `Monday`, ...). Do not use `sns` for this plot. Be sure that your axes are labeled and that your plot is titled.

**Hint:** *Given a series `s`, and an array `coolIndex` that has the same entries as in `s.index`, `s[coolIndex]` will return a copy of the series in the same order as `coolIndex`.*

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

## 3.5 What Day of the Week is more calls?

Is it true that weekdays generally have slightly more calls than Saturday or Sunday? What can you say about the difference?

##### BEGIN SOLUTION


##### END SOLUTION

We can break down into some particular types of events to see their distribution. For example, let's make a bar plot for the OFNS_DESC "HARRASSMENT 2". Which day is the peak for "HARRASSMENT 2"?

This time, use `seaborn` to create a vertical bar plot of the number of total noise violations reported on each day of the week, again ordered by the days of the week starting with Sunday. Do not use `pandas` to plot.

**Hint:** *If you're stuck, use the code for the seaborn plot in above question as a starting point.*

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

### 3.6 

Do you see anything interesting about the distribution of HARRASSMENT 2 calls over a week? Type a short answer below.
##### BEGIN SOLUTION

##### END SOLUTION

### 3.7 More Plots

Let's look at a similar distribution but for a crime we have much more calls data about. In the cell below, create the same plot as you did in previous questions, but now looking at instances of the OFNS_DESC "BURGLARY" (instead of "HARRASSMENT 2"). Use either `pandas` or `seaborn` plotting as you desire.

In [ ]:
### BEGIN SOLUTION


### END SOLUTION



### 3.8 time of events

Now let's look at the CMPLNT_TO_TM column which indicates the time for events. Since it contains hour and minute information, let's extract the hour info and create a new column named `Hour` in the `calls` dataframe. You should save the hour as an `int`. Then plot the frequency of each hour in the table (i.e., `value_counts()`) sorted by the hour of the day (i.e., `sort_index()`).

You will want to look into how to use:

* [Series.str.slice](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.slice.html#pandas.Series.str.slice) to select the substring.
* [Series.astype](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.astype.html) to change the type.

**Hint:** *The `str` helper member of a series can be used to grab substrings.  For example, `calls["CMPLNT_TO_TM"].str.slice(3,5)` returns the minute of each hour of the `CMPLNT_TO_TM`.*

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

Create a pandas bar plot showing the number of BURGLARY crimes committed at each hour of the day. Use the labels
* ax.set_xlabel("Hour of the Day")
* ax.set_ylabel("Number of Calls")
* ax.set_title("Number of Calls Reporting Fraud For Each Day of the Week");

In [ ]:
### BEGIN SOLUTION


### END SOLUTION

### 3.9 More plots

In the cell below, create a seaborn plot of the same data. Again, make sure you provide axes labels and a title for your plot.

In [ ]:
### BEGIN SOLUTION

### END SOLUTION

### 3.8 Spike in burglary?

According to your plots, is there a spike in calls reporting BURGLARY at any particular time? If so, Do you trust that this spike is legitimate, or could there be an issue with our data? Explain your reasoning in 1-2 sentences below.

#### BEGIN SOLUTION


#### END SOLUTION

In the cell below, we generate a boxplot which examines the hour of day of each crime broken down by the `OFNS_DESC` value.  To construct this plot we used the [DataFrame.boxplot](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.boxplot.html) documentation.

In [ ]:
calls.boxplot(column="Hour", by='OFNS_DESC', rot=90);

While the pandas boxplot is informative, we can use seaborn to create a more visually-appealing plot. Using seaborn, regenerate a better box plot. See either the textbook (https://www.textbook.ds100.org/ch/06/viz_quantitative.html) or the [seaborn boxplot documentation](https://seaborn.pydata.org/generated/seaborn.boxplot.html).

Looking at your plot, which crime type appears to have the largest interquartile range? Put your results into `answer` as a string.

In [ ]:
# Todo: Make a boxplot with seaborn
### BEGIN SOLUTION
ax = * Your code here *
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
ax.set_title("Distributions of Calls Over Hours in a Day, Grouped by Crime");
answer = "Your Answer here"
### END SOLUTION

### 3.9 - Visualization of crimes on a Map of NYC
finally we attempt to visualize the crimes committed in NYC on a Map. First we need to installing some mapping software. run the cell below to install folium package for mapping software.

In [ ]:
!pip install --upgrade folium

In [ ]:
### Plot the maps for BURGLARY in NYC
### if it takes too much time or map does not show up, try plotting a subset of the data set of for a specific crime

### BEGIN SOLUTION


### END SOLUTION

## Congratulations !!!

<div class="alert alert-block alert-info">
<h2>Submission Instructions</h2> 
<b> File Name:</b> Please name the file as your_section_your_netID_Lab3.jpynb<br>
<b> Submit To: </b> Canvas &rarr; Assignments &rarr; Lab3 <br>
<b>Warning:</b> Failure to follow directions may result in loss of points.<br>
</div>

Credits: Josh Hug, and Berkeley Data Science Group for their contributions to the original version.